# Setup

In [ ]:
try:
  import google.colab
  IN_COLAB = True
  print("Running as a Colab notebook")
  from google.colab import drive
  drive.mount('/content/gdrive')
  %cd /content/gdrive/MyDrive/AI-ML-Stuff/Dissertation/work/Transformer-Masking
  %pip install scikit-learn
  %pip install transformer_lens
except:
  IN_COLAB = False
  print("Running as a Jupyter notebook - intended for development only!")

Running as a Colab notebook
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/AI-ML-Stuff/Dissertation/work/Transformer-Masking
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
pio.renderers.default = "notebook_connected" # or use "browser" if you want plots to open with browser
import torch as t
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import einops
from fancy_einsum import einsum
from typing import List, Optional, Callable, Tuple, Union
import functools
from tqdm import tqdm
from IPython.display import display
import random
import os
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.utils import shuffle
import json


from transformer_lens.hook_points import HookPoint
from transformer_lens import utils, HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

# Saves computation time, since we don't need it for the contents of this notebook
t.set_grad_enabled(False)

I want to produce datasets which comprise of simple questions, such that the questions have some feature as a similarity. We can then also produce a dataset which incorporates both of these features. The idea will then be to compare the activations of these datasets, so we can make some kind of causal prediction about the activations.

In [ ]:
gpt2_small = HookedTransformer.from_pretrained("gpt2-small").cuda()


Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-small into HookedTransformer
Moving model to device:  cuda


In [ ]:
gpt2_small.generate("Repeat the string 'a f q'. a f", max_new_tokens=10, temperature=0, prepend_bos=True)

  0%|          | 0/10 [00:00<?, ?it/s]

"Repeat the string 'a f q'. a f q is a string.\n\nThe following example"

In [ ]:
gpt2_small.generate("Jimmy, James, Beth and Ben were all at a party. Ben jumped over Oscar. Q: Who was in the air? A:", max_new_tokens=10, temperature=0, prepend_bos=True)

  0%|          | 0/10 [00:00<?, ?it/s]

'Jimmy, James, Beth and Ben were all at a party. Ben jumped over Oscar. Q: Who was in the air? A: Jimmy, James, Beth and Ben were all at'

In [ ]:
gpt2_large = HookedTransformer.from_pretrained("gpt2-large").cuda()


Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-large into HookedTransformer
Moving model to device:  cuda


In [ ]:
gpt2_xl

HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (pos_embed): PosEmbed()
  (hook_pos_embed): HookPoint()
  (blocks): ModuleList(
    (0-47): 48 x TransformerBlock(
      (ln1): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln2): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
      )
      (mlp): MLP(
        (hook_pre): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_attn_out): HookPoint()
      (hook_mlp_out): HookPoint()
      (hook_resid_pre): HookPoint()
      (hook_resid_mid): HookPoint()
   

In [ ]:
gpt2_xl = HookedTransformer.from_pretrained("gpt2-xl").cuda()


Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-xl into HookedTransformer
Moving model to device:  cuda


In [ ]:
testo="""In the enchanting land of Florabella, Princess Anastasia discovered a mysterious garden hidden behind the castle. She confided in her dearest friend, Penelope, and her trusted advisor, Archibald, to explore it. Together, they deciphered an ancient prophecy about the legendary warrior, Maximilian. The prophecy told of a mystical artifact, the Heart of Elizadora, which would awaken Maximilian's true powers. They searched high and low until they found it. Princess Anastasia held the Heart of Elizadora high and whispered the incantation. Before their eyes, the spirit of Maximilian materialized, thanking Anast"""

In [ ]:
gpt2_xl.generate(testo2, max_new_tokens=10, temperature=0, prepend_bos=True)

  0%|          | 0/10 [00:00<?, ?it/s]

"\nIn the quaint village of Meadowbrook, the Thompson family was renowned for their artistry.\nHarold Thompson, the patriarch, was an accomplished painter.\nHis wife, Evelyn Thompson, was a talented sculptor.\nThey had two children, Eliza Thompson and Jonathan Thompson.\nEliza had an affinity for ballet, while Jonathan pursued photography.\nEvelyn's brother, Robert Harrison, often visited them and was known for his comical anecdotes.\n\nThe full name of Eliza's daughter is Evelyn Thompson.\n\nEvelyn was"

In [ ]:
gpt2_small.generate("Jimmy, James, Beth and Ben were all at a party. Ben jumped over Oscar. Q: Who was in the air? A:", max_new_tokens=10, temperature=0, prepend_bos=True)

In [ ]:
gpt_neo_27B = HookedTransformer.from_pretrained("gpt-neo-2.7B").cuda()

Using pad_token, but it is not set yet.


Loaded pretrained model gpt-neo-2.7B into HookedTransformer
Moving model to device:  cuda


gpt-neo-2.7B

In [ ]:
testo3 = """ """

In [ ]:
template = f"""
In the quaint village of Meadowbrook, the Thompson family was renowned for their artistry.
{patriarch}, the patriarch, was an accomplished painter.
His wife, {matriarch} Thompson, was a talented sculptor.
They had two children, {child1} and {child2}.
{matriarch}'s brother, {brother}, often visited them and was known for his comical anecdotes.

The name of {matriarch}'s {relation} is"""

NameError: ignored

In [ ]:
family_names = {
    "patriarch": ["Robert", "John", "William", "James", "Richard", "Charles", "Joseph", "Michael", "David", "Thomas"],
    "matriarch": ["Margaret", "Elizabeth", "Susan", "Mary", "Patricia", "Linda", "Barbara", "Jennifer", "Jessica", "Sarah"],
    "child1": ["Emily", "Sophia", "Olivia", "Emma", "Ava", "Isabella", "Mia", "Abigail", "Madison", "Charlotte"],
    "child2": ["Ethan", "Noah", "Liam", "Alexander", "Aiden", "Lucas", "Benjamin", "Mason", "Elijah", "Jackson"],
    "brother": ["Daniel", "Christopher", "Matthew", "Andrew", "Nicholas", "Joshua", "Anthony", "Ryan", "Brandon", "Kevin"]
}

In [ ]:
import random

def dataset_creator(number, seed):
  random.seed(seed)
  family_names = {
      "husband": ["Robert", "John", "William", "James", "Richard", "Charles", "Joseph", "Michael", "David", "Thomas"],
      "matriarch": ["Margaret", "Elizabeth", "Susan", "Mary", "Patricia", "Linda", "Barbara", "Jennifer", "Jessica", "Sarah"],
      "child": ["Emily", "Sophia", "Olivia", "Emma", "Ava", "Isabella", "Mia", "Abigail", "Madison", "Charlotte"],
      "cousin": ["Ethan", "Noah", "Liam", "Alexander", "Aiden", "Lucas", "Benjamin", "Mason", "Elijah", "Jackson"],
      "brother": ["Daniel", "Christopher", "Matthew", "Andrew", "Nicholas", "Joshua", "Anthony", "Ryan", "Brandon", "Kevin"]
  }

  selected_names = {
      key: random.choice(value)
      for key, value in family_names.items()
  }

  template = f"""
  The Thompson family was renowned for their artistry.
  {{matriarch}} was the matriarch.
  Her husband was {{husband}}.
  They had one child, {{child}}.
  {{matriarch}} had a cousin, {{cousin}}
  {{matriarch}}'s brother was called {{brother}}.

  The name of {{husband}}'s wife was {{matriarch}}.
  The name of {{matriarch}}'s {{relation}} was"""

  if number == 2 or number == 3:
    # Add a relation to selected names
    selected_names["relation"] = "brother"
  else:
    relations = ["brother", "husband", "cousin", "child"]
    selected_names["relation"] = random.choice(relations)

  # Fix the output name
  if number == 1 or number == 3:
    relation = selected_names["relation"]
    selected_names[relation] = "Taylor"

  # Format the template string with the selected names
  story = template.format(**selected_names)

  # Return the generated story
  return story

In [ ]:
dataset_1 = []
for i in range(500):
  taylor_sentence = dataset_creator(1, i)
  dataset_1.append(taylor_sentence)

In [ ]:
dataset_1[29]

"\n  The Thompson family was renowned for their artistry.\n  Elizabeth was the matriarch.\n  Her husband was David.\n  They had one child, Isabella.\n  Elizabeth had a cousin, Taylor\n  Elizabeth's brother was called Kevin.\n\n  The name of David's wife was Elizabeth.\n  The name of Elizabeth's cousin was"

In [ ]:
dataset_2 = []
for i in range(500):
  brother_sentence = dataset_creator(2, i)
  dataset_2.append(brother_sentence)

In [ ]:
dataset_2[29]

"\n  The Thompson family was renowned for their artistry.\n  Elizabeth was the matriarch.\n  Her husband was David.\n  They had one child, Isabella.\n  Elizabeth had a cousin, Jackson\n  Elizabeth's brother was called Kevin.\n\n  The name of David's wife was Elizabeth.\n  The name of Elizabeth's brother was"

In [ ]:
dataset_3 = []
for i in range(500):
  brother_sentence = dataset_creator(3, i)
  dataset_3.append(brother_sentence)

In [ ]:
dataset_3[0]

"\n  The Thompson family was renowned for their artistry.\n  Barbara was the matriarch.\n  Her husband was Joseph.\n  They had one child, Emily.\n  Barbara had a cousin, Aiden\n  Barbara's brother was called Taylor.\n\n  The name of Joseph's wife was Barbara.\n  The name of Barbara's brother was"

In [ ]:
gpt2_large.generate(dataset_3[0], max_new_tokens=10, temperature=0, prepend_bos=True)

  0%|          | 0/10 [00:00<?, ?it/s]

"\n  The Thompson family was renowned for their artistry.\n  Barbara was the matriarch.\n  Her husband was Joseph.\n  They had one child, Emily.\n  Barbara had a cousin, Aiden\n  Barbara's brother was called Taylor.\n\n  The name of Joseph's wife was Barbara.\n  The name of Barbara's brother was Joseph.\n  Joseph's wife was named Barbara"

In [ ]:
import json

In [ ]:
with open("family_dataset1_big.json", "w") as file:
  json.dump(dataset_1, file)

In [ ]:
with open("family_dataset2_big.json", "w") as file:
  json.dump(dataset_2, file)

In [ ]:
with open("family_dataset3_big.json", "w") as file:
  json.dump(dataset_3, file)

In [ ]:
same_completion_dataset = []

In [ ]:
testo = (
"""Jessica went to sit in her rocking chair. Today was her birthday
and she was turning 80. Her granddaughter Annie was coming
over in the afternoon and Jessica was very excited to see
her. Her daughter Melanie and Melanie’s husband Josh were
coming as well. Jessica had . . .
Q: Who is Jessica's granddaughter?
A: John
B: Annie
C: Jessica
D: Melanie

Answer: B

Q: Who had a birthday?
A: John
B: Annie
C: Jessica
D: Melanie

Answer: C

Q: What is Jessica's name?
A: John
B: Annie
C: Jessica
D: Melanie

Answer:"""
)

In [ ]:
gpt2_xl.generate(testo, max_new_tokens=10, temperature=0, prepend_bos=True)

  0%|          | 0/10 [00:00<?, ?it/s]

"Jessica went to sit in her rocking chair. Today was her birthday\nand she was turning 80. Her granddaughter Annie was coming\nover in the afternoon and Jessica was very excited to see\nher. Her daughter Melanie and Melanie’s husband Josh were\ncoming as well. Jessica had...\nQ: Who is Jessica's granddaughter?\nA: John\nB: Annie\nC: Jessica\nD: Melanie\n\nAnswer: B\n\nQ: Who had a birthday?\nA: John\nB: Annie\nC: Jessica\nD: Melanie\n\nAnswer: C\n\nQ: What is Jessica's name?\nA: John\nB: Annie\nC: Jessica\nD: Melanie\n\nAnswer: D\n\nQ: What is Jessica's favorite"

In [ ]:
repetition_text = (
"""Repeat the string 'a x w'. a x

Q: What comes next?

1: b
2: r
3: q
4: w
Answer: w

Repeat the string 'h r c'. h r

Q: What comes next?

1: a
2: r
3: c
4: w
Answer: c

Repeat the string 's q p'. s q

Q: What comes next?

1: p
2: w
3: e
4: i
Answer:"""
)

In [ ]:
gpt_neo_27B.generate(first_letter_text, max_new_tokens=10, temperature=0, prepend_bos=True)

  0%|          | 0/10 [00:00<?, ?it/s]

'Q: Which word starts with b?\n\n1: bug\n2: rhino\n3: quest\n4: welsh\nAnswer: 1\n\nQ: Which word starts with d?\n\n1: cat\n2: rabbit\n3: dog\n4: horse\nAnswer: 3\n\nQ: Which word starts with b?\n\n1: cat\n2: boffin\n3: horse\n4: rhino\nAnswer: 1\n\nQ: Which word starts with d'

In [ ]:
gpt2_xl.generate(short_text, max_new_tokens=5, temperature=0, prepend_bos=True)

  0%|          | 0/5 [00:00<?, ?it/s]

'\nQ: Which word starts with b?\n\n1: cat\n2: boffin\n3: horse\n4: rhino\nAnswer: b\n\nQ:'

In [ ]:
first_letter_text = (
"""Q: Which word starts with b?

1: bug
2: rhino
3: quest
4: welsh
Answer: 1

Q: Which word starts with d?

1: cat
2: rabbit
3: dog
4: horse
Answer: 3

Q: Which word starts with b?

1: cat
2: boffin
3: horse
4: rhino
Answer:"""
)

In [ ]:
short_text = (
"""
Q: Which word starts with b?

1: cat
2: boffin
3: horse
4: rhino
Answer:"""
)